In [1]:
# Load Python Libraries
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

In [2]:
# Specify paths to IR data and terrain data
IRpath = '/squall/ldgrant/INCUS_plots/MergIR/data/Amazon_case4/'
Tfile = '/avalanche/pmarin/INCUS/MERGIR/elev.0.25-deg.nc'
files = sorted(glob.glob(IRpath+'*.nc4'))

# Specify path for saving figures
savepath = '/squall/ldgrant/INCUS_plots/MergIR/plots/Amazon_case4/'
if not os.path.exists(savepath):
   os.makedirs(savepath)

In [3]:
# Load Terrain Data
ter_data = Dataset(Tfile)
#print(ter_data['data'])
#print(ter_data['lat'])
print(len(files))
print(files[0])
print(files[1])
print(files[-1])

96
/squall/ldgrant/MergIR/data/Amazon_case4/merg_2014081400_4km-pixel.nc4.nc4
/squall/ldgrant/MergIR/data/Amazon_case4/merg_2014081401_4km-pixel.nc4.nc4
/squall/ldgrant/MergIR/data/Amazon_case4/merg_2014081723_4km-pixel.nc4.nc4


In [4]:
# Specify lat/lon bounds
xlims = [-85,-30]
ylims = [-30,15]

# Specify contour levels for different variables
lvls = np.arange(200,300.1,5) # Tb levels, K
tlvls = np.arange(500,5001,1000) # terrain levels, m

# Specify whether to plot a city and/or radar range ring
plotcity = 1
plotradar = 1
# specify lat/lon of scatter, and radius of ring
citylat = -3.12 # Manaus
citylon = -60.02
radarrad = 1.8 # this is ~200 km radius converted to deg lat/lon, for Manaus

i = 0
# Loop throgh files and plot Tb and terrain on a map
#for f in np.arange(87,88,1):
for f in np.arange(0,len(files),1):
    
    # Read in brightness temperature data
    data = Dataset(files[f])
    #print(data)
    Tb = data['Tb'][:]
    lat = data['lat'][:]
    lon = data['lon'][:]
    time = data['time'][:]
    #print(time)
    # Get date string from filename
    datestr = os.path.basename(files[f])[5:15]
    #print(datestr)
    print(f)
    
    for h in np.arange(0,2):
        fig,ax = plt.subplots(1,1,subplot_kw={'projection': ccrs.PlateCarree()},figsize=[11,8])
        
        if h == 0:
            timestr = '00'
        elif h==1:
            timestr = '30'

        ax.add_feature(cfeature.COASTLINE) # Add coastlines
        ax.add_feature(cfeature.BORDERS) # Add boarders 
        ax.add_feature(cfeature.RIVERS) # Add Rivers
        ax.add_feature(cfeature.LAKES) # Add Rivers

        # Contourf brightness temperatures [0 value in first axis specifies grabbing either XX00 (HHMM) data, as opposed to data on the on the XX30 (HHMM) data
        # changed to h in order to plot both 00 and 30 min data
        a = ax.contourf(lon,lat,Tb[h,:,:],levels=lvls,extend='both',transform = ccrs.PlateCarree())

        # Contour terrain data
        ax.contour(ter_data['lon'][:],ter_data['lat'][:],ter_data['data'][0,:,:],levels=tlvls,colors='brown',transform = ccrs.PlateCarree())
        
        # add scatter point for a city, and a radar range ring if desired
        if plotcity == 1:
            ax.scatter(citylon,citylat,color='white',s=20,transform = ccrs.PlateCarree())
        if plotradar == 1:
            circ=patches.Circle( (citylon,citylat), radarrad, edgecolor='white', fill = False, transform = ccrs.PlateCarree() )
            ax.add_patch(circ)


        # Add grid lines
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='-')

        # Specify title and lat/lon bounds
        ax.set_title('Tb at '+datestr[0:4]+'-'+datestr[4:6]+'-'+datestr[6:8]+' '+datestr[8:10]+':'+timestr+' Z')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')

        # Set x and y axis limits
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)

        # Plot colorbar
        cbar = plt.colorbar(a,ax=ax)        
        cbar.ax.set_ylabel('Brightness Temperature (K)')
        
        plt.tight_layout()

        #Create filename for saving
        #savename = 'MERGIR_Tb_'+datestr+'_'+'Lon_'+str(xlims[0])+str(xlims[1])+'_Lat_'+str(ylims[0])+str(ylims[1])+'.png'
        savename = 'MERGIR_Tb_'+datestr+timestr+'.png'
        # Save Figure
        print('Saving '+savepath+savename)
        plt.savefig(savepath+savename)

        plt.close(fig) # Close figure - prevents it from appearing in Jupyterlab window
        
    data.close() # Close netcdf file

0
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140000.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140030.png
1
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140100.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140130.png
2
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140200.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140230.png
3
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140300.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140330.png
4
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140400.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140430.png
5
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140500.png
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140530.png
6
Saving /squall/ldgrant/MergIR/plots/Amazon_case4/MERGIR_Tb_201408140600.pn